# **Mildew Detector**

---

# Data Collection Notebook

---

## Objectives

* Fetch data from Kaggle and save as raw data
* Check data for non-image files
* Split data into Test, Train and Validation sets

## Inputs

* Kaggle JSON file - authentication token

## Outputs

```
. 
├── inputs 
│   └──mildew-dataset 
│      └──cherry-leaves                                     
│           ├── test
│           │   ├── healthy
│           │   └── powdery_mildew                   
│           ├── train
│           │   ├── healthy
│           │   └── powdery_mildew          
│           └── validation
│               ├── healthy
│               └── powdery_mildew/
```

## Additional Comments

* This notebook goes through the standard steps to import, clean and prepare the data, in preparation for machine learning.

---

# Change working directory

---

The notebooks are stored in a subfolder, so the following steps change the working directory to the parent folder so the data can be accessed.

* Access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

To make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

# Install Kaggle

---

Install Kaggle to enable the data to be imported.

In [ ]:
# Install Kaggle Package
%pip install kaggle==1.5.12

Run the cell below to change the Kaggle configuration directory to the current working directory and set permissions for the Kaggle authentication JSON.

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

Import the data from Kaggle

In [ ]:
KaggleDatasetPath = "codeinstitute/cherry-leaves"
DestinationFolder = "inputs/cherry-leaf-dataset"
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

In [ ]:
import zipfile
with zipfile.ZipFile(DestinationFolder + '/cherry-leaves.zip', 'r') as zip_ref:
    zip_ref.extractall(DestinationFolder)

os.remove(DestinationFolder + '/cherry-leaves.zip')

---

# Data Preparation

---

Check the data for any files which are not images.

In [ ]:
def remove_non_image_file(my_data_dir):
    image_extension = ('.png', '.jpg', '.jpeg')
    folders = os.listdir(my_data_dir)
    for folder in folders:
        files = os.listdir(my_data_dir + '/' + folder)
        # print(files)
        i = []
        j = []
        for given_file in files:
            if not given_file.lower().endswith(image_extension):
                file_location = my_data_dir + '/' + folder + '/' + given_file
                os.remove(file_location)  # remove non image file
                i.append(1)
            else:
                j.append(1)
                pass
        print(f"Folder: {folder} - has image file", len(j))
        print(f"Folder: {folder} - has non-image file", len(i))

Remove any non image files from the dataset

In [ ]:
remove_non_image_file(my_data_dir='inputs/mildew-dataset/cherry-leaves')

---

# Split into Sets

---

Split the data into train (70%), test (20%) and vaildation (10%) sets.

In [ ]:
import os
import shutil
import random
import joblib


def split_train_validation_test_images(my_data_dir, train_set_ratio, validation_set_ratio, test_set_ratio):

    if train_set_ratio + validation_set_ratio + test_set_ratio != 1.0:
        print("train_set_ratio + validation_set_ratio + test_set_ratio should sum to 1.0")
        return

    # gets classes labels
    labels = os.listdir(my_data_dir)  # it should get only the folder name
    if 'test' in labels:
        pass
    else:
        # create train, test folders with classes labels sub-folder
        for folder in ['train', 'validation', 'test']:
            for label in labels:
                os.makedirs(name=my_data_dir + '/' + folder + '/' + label)

        for label in labels:

            files = os.listdir(my_data_dir + '/' + label)
            random.shuffle(files)

            train_set_files_qty = int(len(files) * train_set_ratio)
            validation_set_files_qty = int(len(files) * validation_set_ratio)

            count = 1
            for file_name in files:
                if count <= train_set_files_qty:
                    # move a given file to the train set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/train/' + label + '/' + file_name)

                elif count <= (train_set_files_qty + validation_set_files_qty):
                    # move a given file to the validation set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/validation/' + label + '/' + file_name)

                else:
                    # move given file to test set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/test/' + label + '/' + file_name)

                count += 1

            os.rmdir(my_data_dir + '/' + label)

In [ ]:
split_train_validation_test_images(my_data_dir=f"inputs/mildew-dataset/cherry-leaves",
                                   train_set_ratio=0.7,
                                   validation_set_ratio=0.1,
                                   test_set_ratio=0.2
                                   )

---

# Push files to Repo

---

If required, uncomment code below and run cell.

In [ ]:
# import os
# try:
#     # create here your folder
#     # os.makedirs(name='')
# except Exception as e:
#     print(e)